***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-8ae7  GCE       3                                             RUNNING  us-central1-a


# Imports & Setup

In [2]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Feb 29 07:16 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'parwuets-101' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh':
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [21]:
parquetFile = spark.read.parquet(*paths)

In [52]:
doc_title_pairs = parquetFile.select("id", "title").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [26]:
bucket_name = 'norse-botany-bucket' 

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [27]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [28]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [29]:
from inverted_index_gcp import InvertedIndex

In [30]:
from nltk.stem.porter import *

In [55]:
dfTitle = doc_title_pairs.toDF()

In [59]:
dfTitle.write.option("header", "true").csv(f'gs://{bucket_name}/indexes/id_title.csv')

In [60]:
df = spark.read.option("header", "true").csv(f'gs://{bucket_name}/indexes/id_title.csv')

In [62]:
df.show()

+--------+--------------------+
|      id|               title|
+--------+--------------------+
|11659683| 1980 in Afghanistan|
|11659685|Memorial Stadium ...|
|11659694|List of the oldes...|
|11659697|Wine/water mixing...|
|11659698|       Howard Andrew|
|11659707|United States Pos...|
|11659752|     Darkest of Days|
|11659771|         Su Sen-yong|
|11659780|           Mark Wade|
|11659785| Heathcote (surname)|
|11659791|       Delbert Baker|
|11659797|  Topes de Collantes|
|11659808|         Bánh da lợn|
|11659839|   Adailton de Jesus|
|11659850|Big Time (Whigfie...|
|11659860| 1982 in Afghanistan|
|11659866|     Milana Terloeva|
|11659868|Causes of the Ind...|
|11659876| 1983 in Afghanistan|
|11659879|Hokuto "Hok" Konishi|
+--------+--------------------+
only showing top 20 rows

